In [1]:
%matplotlib inline
from numpy import *
import os
from os import listdir
from os.path import isfile, join
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cbook as cbook
import time
from scipy.misc import imread
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from random import sample
from numpy import random

import tensorflow as tf

from caffe_classes import class_names

In [2]:
n_classes = 3 # my classes (1000 # ImageNet)
train_x = zeros((1, 227,227,3)).astype(float32)
train_y = zeros((1, n_classes))
xdim = train_x.shape[1:]
ydim = train_y.shape[1]
learning_rate = 0.01
#net_data = load("bvlc_alexnet.npy").item()
net_data = load("bvlc_alexnet.npy").item()
new_net = net_data
batch_size = 20
training_iters = 252
display_step = 1
dropout = 0.9 # Dropout, probability to keep units

In [3]:
# Read a few images for sanity check

im1 = (imread("dog_2002.png")[:,:,:3]).astype(float32)
im1 = im1 - mean(im1)

im2 = (imread("cat_8.jpg")[:,:,:3]).astype(float32)
im2 = im2 - mean(im2)

im3 = (imread("dog_31.jpg")[:,:,:3]).astype(float32)
im3 = im3 - mean(im3)

im4 = (imread("flower_33.jpg")[:,:,:3]).astype(float32)
im4 = im4 - mean(im4)

In [12]:
# (self.feed('data')
#         .conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
#         .lrn(2, 2e-05, 0.75, name='norm1')
#         .max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
#         .conv(5, 5, 256, 1, 1, group=2, name='conv2')
#         .lrn(2, 2e-05, 0.75, name='norm2')
#         .max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
#         .conv(3, 3, 384, 1, 1, name='conv3')
#         .conv(3, 3, 384, 1, 1, group=2, name='conv4')
#         .conv(3, 3, 256, 1, 1, group=2, name='conv5')
#         .fc(4096, name='fc6')
#         .fc(4096, name='fc7')
#         .fc(1000, relu=False, name='fc8') --> change to 3 outputs
#         .softmax(name='prob'))


def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    assert c_i%group==0
    assert c_o%group==0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)   
    
    if group==1:
        conv = convolve(input, kernel)
    else:
        input_groups = tf.split(3, group, input)
        kernel_groups = tf.split(3, group, kernel)
        output_groups = [convolve(i, k) for i,k in zip(input_groups, kernel_groups)]
        conv = tf.concat(3, output_groups)
    return  tf.reshape(tf.nn.bias_add(conv, biases), [-1]+conv.get_shape().as_list()[1:])


# Create model
def conv_net(x, weights, biases, dropout):
    
    #conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
    k_h = 11; k_w = 11; c_o = 96; s_h = 4; s_w = 4
    conv1_in = conv(x, weights['conv1W'], biases['conv1b'], k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
    conv1 = tf.nn.relu(conv1_in)

    #lrn(2, 2e-05, 0.75, name='norm1')
    radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
    lrn1 = tf.nn.local_response_normalization(conv1, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)

    #max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
    k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
    maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)
    
    #conv(5, 5, 256, 1, 1, group=2, name='conv2')
    k_h = 5; k_w = 5; c_o = 256; s_h = 1; s_w = 1; group = 2
    conv2_in = conv(maxpool1, weights['conv2W'], biases['conv2b'], k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv2 = tf.nn.relu(conv2_in)
    
    #lrn(2, 2e-05, 0.75, name='norm2')
    radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
    lrn2 = tf.nn.local_response_normalization(conv2, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
    
    #max_pool(3, 3, 2, 2, padding='VALID', name='pool2')                                                  
    k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
    maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

    #conv(3, 3, 384, 1, 1, name='conv3')
    k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 1
    conv3_in = conv(maxpool2, weights['conv3W'], biases['conv3b'], k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv3 = tf.nn.relu(conv3_in)
    
    #conv(3, 3, 384, 1, 1, group=2, name='conv4')
    k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 2
    conv4_in = conv(conv3, weights['conv4W'], biases['conv4b'], k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv4 = tf.nn.relu(conv4_in)
    
    #conv(3, 3, 256, 1, 1, group=2, name='conv5')
    k_h = 3; k_w = 3; c_o = 256; s_h = 1; s_w = 1; group = 2
    conv5_in = conv(conv4, weights['conv5W'], biases['conv5b'], k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv5 = tf.nn.relu(conv5_in)
    
    #max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
    k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
    maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)
    
    #fc(4096, name='fc6')
    fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [-1, int(prod(maxpool5.get_shape()[1:]))]), weights['fc6W'], biases['fc6B'])
    fc6 = tf.nn.dropout(fc6, dropout)
    
    #fc(4096, name='fc7')
    fc7 = tf.nn.relu_layer(fc6, weights['fc7W'], biases['fc7B'])
    fc7 = tf.nn.dropout(fc7, dropout)
    
    #fc(1000, relu=False, name='fc8')
    fc8 = tf.nn.xw_plus_b(fc7, weights['fc8W'], biases['fc8B'])
    
    #softmax(name='prob'))
    #!prob = tf.nn.softmax(fc8)
    
    #return prob
    return fc8

def getfc8w(weights):
    return weights['fc8W']
def getfc7w(weights):
    return weights['fc7W']
def getfc6w(weights):
    return weights['fc6W']
def getfc8b(biases):
    return biases['fc8B']
def getfc7b(biases):
    return biases['fc7B']
def getfc6b(biases):
    return biases['fc6B']


def getFeat(x):
    return prob(x,weights,biases)

In [13]:
# Store layers weight & bias
weights = {    
    'conv1W': tf.Variable(net_data["conv1"][0], name="c1w"),
    'conv2W': tf.Variable(net_data["conv2"][0], name="c2w"),
    'conv3W': tf.Variable(net_data["conv3"][0], name="c3w"),
    'conv4W': tf.Variable(net_data["conv4"][0], name="c4w"),
    'conv5W': tf.Variable(net_data["conv5"][0], name="c5w"),
    'fc6W': tf.Variable(net_data["fc6"][0], name="fc6w"),
    'fc7W': tf.Variable(net_data["fc7"][0], name="fc7w"),
    'fc8W': tf.Variable(tf.random_normal([4096, n_classes]),trainable=True, name="fc8w")
}

biases = {    
    'conv1b': tf.Variable(net_data["conv1"][1], name="c1b"),
    'conv2b': tf.Variable(net_data["conv2"][1], name="c2b"),
    'conv3b': tf.Variable(net_data["conv3"][1], name="c3b"),
    'conv4b': tf.Variable(net_data["conv4"][1], name="c4b"),
    'conv5b': tf.Variable(net_data["conv5"][1], name="c5b"),
    'fc6B': tf.Variable(net_data["fc6"][1], name="fc6b"),
    'fc7B': tf.Variable(net_data["fc7"][1], name="fc7b"),
    'fc8B': tf.Variable(tf.random_normal([n_classes]),trainable=True, name="fc8b")
}


In [14]:
x = tf.placeholder(tf.float32, (None,) + xdim) # None = number of input images
y = tf.placeholder(tf.float32, [None,ydim])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

# Construct model
prob = conv_net(x, weights, biases, keep_prob)

# analysis
currentfc8w = getfc8w(weights)
currentfc7w = getfc7w(weights)
currentfc6w = getfc6w(weights)
currentfc8b = getfc8b(biases)
currentfc7b = getfc7b(biases)
currentfc6b = getfc6b(biases)

feat = tf.nn.softmax(prob)

# Loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prob, y))
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
opt_vars = [v for v in tf.trainable_variables() if v.name.startswith("fc8")]
#opt_vars = [v for v in tf.trainable_variables() if v.name.startswith("fc")]
#print shape(opt_vars)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, var_list = opt_vars)

# Evaluation
correct_pred = tf.equal(tf.argmax(prob, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
def next_batch(batch_size):
    ImgPath = "/ImgDir/"
    files = [f for f in listdir(ImgPath) if isfile(join(ImgPath, f))]
    files = sample(files, len(files))
    batch_x = np.ndarray([batch_size,227, 227, 3])
    batch_y = np.zeros((batch_size, 3))
    i = 0
    for fname in files:
        img = (imread(join(ImgPath, fname))[:,:,:3]).astype(float32)
        img = img - mean(img)
        #img = np.reshape(img,(1,227*227*3))        
        batch_x[i] = img
        if "cat" in fname:
            batch_y[i][0] = 1
        if "dog" in fname:
            batch_y[i][1] = 1
        if "flower" in fname:
            batch_y[i][2] = 1
        i+=1
        if i == batch_size:
            yield (batch_x, batch_y)
            batch_x = np.ndarray([batch_size,227, 227, 3])
            batch_y = np.zeros((batch_size, 3))
            i=0            
#     if len(batch_x) > 0:
#         yield (batch_x, batch_y)


In [16]:
# Fine tune network
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step = 1    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = next_batch(batch_size).next()        
        # Run optimization op (backprop)        
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})                                
                
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            cost, acc = sess.run([loss, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(cost) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)                          
        step += 1
    print "Optimization Finished!"
    
    output = sess.run(feat, feed_dict = {x:[im1,im2,im3,im4], keep_prob: 1.})
    output2 = sess.run(prob, feed_dict = {x:[im1,im2,im3,im4], keep_prob: 1.})
    print np.argmax(output,1)
    print np.argmax(output2,1)
    
    new_net["fc6"][0] = sess.run(currentfc6w)
    new_net["fc6"][1] = sess.run(currentfc6b)
    new_net["fc7"][0] = sess.run(currentfc7w)
    new_net["fc7"][1] = sess.run(currentfc7b)
    new_net["fc8"][0] = sess.run(currentfc8w)
    new_net["fc8"][1] = sess.run(currentfc8b)

np.save("fine_tuned_net.npy", new_net)

    # Calculate accuracy for 256 mnist test images
    #print "Testing Accuracy:", \
    #   sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
    #                                y: mnist.test.labels[:256],
    #                                keep_prob: 1.})    

Iter 20, Minibatch Loss= 80.041885, Training Accuracy= 0.30000
Iter 40, Minibatch Loss= 60.920509, Training Accuracy= 0.45000
Iter 60, Minibatch Loss= 68.393311, Training Accuracy= 0.30000
Iter 80, Minibatch Loss= 63.801270, Training Accuracy= 0.40000
Iter 100, Minibatch Loss= 42.111069, Training Accuracy= 0.60000
Iter 120, Minibatch Loss= 16.665831, Training Accuracy= 0.70000
Iter 140, Minibatch Loss= 11.684101, Training Accuracy= 0.75000
Iter 160, Minibatch Loss= 13.826100, Training Accuracy= 0.70000
Iter 180, Minibatch Loss= 7.467756, Training Accuracy= 0.80000
Iter 200, Minibatch Loss= 1.694727, Training Accuracy= 0.95000
Iter 220, Minibatch Loss= 7.335664, Training Accuracy= 0.85000
Iter 240, Minibatch Loss= 9.339417, Training Accuracy= 0.85000
Optimization Finished!
[1 0 1 2]
[1 0 1 2]


In [17]:
print output
print output2
print np.argmax(output,1)
print np.argmax(output2,1)

[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[ -34.99688339   85.19901276 -222.18301392]
 [ 134.80635071 -186.3434906  -173.7820282 ]
 [-123.13925171  107.79956818  -89.42668915]
 [-105.69404602  -75.42551422   44.12779617]]
[1 0 1 2]
[1 0 1 2]
